In [1]:
# !wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
!wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json

--2025-02-13 02:58:44--  http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json
Resolving curtis.ml.cmu.edu (curtis.ml.cmu.edu)... 128.2.204.193
Connecting to curtis.ml.cmu.edu (curtis.ml.cmu.edu)|128.2.204.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47454698 (45M) [application/json]
Saving to: ‘hotpot_dev_fullwiki_v1.json’

hotpot_dev_fullwiki 100%[===================>]  45.26M  76.0MB/s    in 0.6s    

2025-02-13 02:58:45 (76.0 MB/s) - ‘hotpot_dev_fullwiki_v1.json’ saved [47454698/47454698]



In [2]:
import json
import pandas as pd
import random

from typing import List, Tuple

In [3]:
def generate_context(input: Tuple[str, List[str]]):
    title, context = input
    return f"{title}\n{''.join(context)}"


def generate_contexts(contexts: List[Tuple[str, List[str]]]):
    context_strings = "\n\n".join([generate_context(context) for context in contexts])
    return f"Contexts:\n\n{context_strings}"


def generate_question(question: str):
    return f"Question: {question}"

In [4]:
df_train_raw = json.load(open("hotpot_train_v1.1.json", "rb"))

In [4]:
random.shuffle(df_train_raw)

df_train = df_train_raw[:-1100]
df_val = df_train_raw[-1100:-1000]
df_test = df_train_raw[-1000:]

df_train = pd.DataFrame(df_train)
df_val = pd.DataFrame(df_val)
df_test = pd.DataFrame(df_test)

df_train["context"] = df_train["context"].apply(generate_contexts)
df_val["context"] = df_val["context"].apply(generate_contexts)
df_test["context"] = df_test["context"].apply(generate_contexts)

df_train["question"] = df_train["question"].apply(generate_question)
df_val["question"] = df_val["question"].apply(generate_question)
df_test["question"] = df_test["question"].apply(generate_question)

df_train["source"] = df_train.apply(
    lambda row: f"{row['context']}\n\n{row['question']}", axis=1
)
df_val["source"] = df_val.apply(
    lambda row: f"{row['context']}\n\n{row['question']}", axis=1
)
df_test["source"] = df_test.apply(
    lambda row: f"{row['context']}\n\n{row['question']}", axis=1
)

In [5]:
df_train = df_train.rename(columns={"_id": "id", "answer": "target"})
df_val = df_val.rename(columns={"_id": "id", "answer": "target"})
df_test = df_test.rename(columns={"_id": "id", "answer": "target"})

In [6]:
df_train = df_train[["id", "source", "target"]]
df_val = df_val[["id", "source", "target"]]
df_test = df_test[["id", "source", "target"]]

In [7]:
df_train.to_csv("train.csv", index=False)
df_val.to_csv("val.csv", index=False)
df_test.to_csv("test.csv", index=False)